In [1]:
"""
BIOINFORMATICS: LAB07
@author: Irene Benedetto
"""


import tensorflow as tf
import numpy as np
from tqdm import tqdm
from models import *


def normalize(data, label):
    data = tf.cast(data, tf.float32)/255.0
    data = tf.expand_dims(data, axis=-1)
    return data, label

In [2]:
mnist = tf.keras.datasets.mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

images = tf.concat([train_images, test_images], axis = 0)
labels = tf.concat([train_images, test_images], axis = 0)
N_EPOCHS = 20
BATCH_SIZE = 64

train_ds = tf.data.Dataset.from_tensor_slices((images, labels))

train_ds = train_ds.map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_ds = train_ds.shuffle(buffer_size=1024).batch(BATCH_SIZE).prefetch(BATCH_SIZE)

input_shape = list(train_images.shape)[1:]
output_shape = len(set(train_labels))
latent_dim = 128

print(f'Training set: {input_shape}')
print(f'Number of classes: {output_shape}')

input_shape = list(train_images.shape)[1:]
output_shape = len(set(train_labels))
latent_dim = 128

print(f'Training set: {input_shape}')
print(f'Number of classes: {output_shape}')



Training set: [28, 28]
Number of classes: 10
Training set: [28, 28]
Number of classes: 10


In [3]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
d_metrics = tf.keras.metrics.SparseCategoricalAccuracy(name='discriminator_accuracy')
g_metrics = tf.keras.metrics.SparseCategoricalAccuracy(name='generator_accuracy')
metrics = tf.keras.metrics.SparseCategoricalAccuracy(name='gan_accuracy')

g_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0003)
d_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0003)

discriminator = Discriminator(input_shape=input_shape + [1], output_shape=1)
discriminator.compile(loss, d_optimizer, d_metrics)

generator = Generator(latent_dim=latent_dim)
generator.compile(loss, g_optimizer, g_metrics)
gan = GAN(generator=generator, discriminator=discriminator, latent_dim=latent_dim, BATCH_SIZE=BATCH_SIZE)

gan.compile(loss, d_optimizer, g_optimizer, metrics)

for epoch in range(N_EPOCHS):

    losses = {"discriminator_loss": 0, "generator_loss": 0}
    print(f'Epoch {epoch + 1}/{N_EPOCHS}')
    for x, y in tqdm(train_ds):
        history = gan.train_step((x, y))
        losses['discriminator_loss'] += history['discriminator_loss']
        losses['generator_loss'] += history['generator_loss']

    print(f"Discriminator loss: {losses['discriminator_loss']}")
    print(f"Generator loss: {losses['generator_loss']}")

    random_latent_vectors = tf.random.normal(shape=(1, latent_dim))
    generated_images = gan.generator(random_latent_vectors)
    generated_images *= 255
    generated_images.numpy()

    img = tf.keras.preprocessing.image.array_to_img(generated_images[0])
    img.save("synthetic_img_{epoch}.png".format(epoch=epoch))



  0%|          | 0/1094 [00:00<?, ?it/s]

Epoch 1/20


  0%|          | 1/1094 [00:00<02:44,  6.64it/s]

Discriminator loss: 537.7638549804688
Generator loss: 1287.8514404296875
Epoch 2/20


  0%|          | 1/1094 [00:00<02:25,  7.54it/s]

...

Discriminator loss: 822.3173828125
Generator loss: 839.575927734375
Epoch 19/20


  0%|          | 1/1094 [00:00<02:09,  8.46it/s]

Discriminator loss: 751.70458984375
Generator loss: 902.0648803710938
Epoch 20/20


100%|██████████| 1094/1094 [00:29<00:00, 36.88it/s]

Discriminator loss: 743.4371337890625
Generator loss: 958.8997192382812
